### Analysis
#1. Charter schools out perform (have higher passing percentages than) district schools, as the top school performers by type are all charter and the bottom performers are all district. 
#2. Per student budget plays little role in performance (student passing percentage), as top school performers' per student budgets are less than bottom school performers' per student budgets. 

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
total_schools = len(school_data_complete["school_name"].unique())
total_schools

15

In [3]:
total_students = len(school_data_complete["Student ID"])
total_students

39170

In [4]:
total_budget = float(school_data["budget"].sum())
total_budget

24649428.0

In [5]:
avg_math_score = float(school_data_complete["math_score"].mean())
avg_math_score

78.98537145774827

In [6]:
avg_reading_score = float(school_data_complete["reading_score"].mean())
avg_reading_score

81.87784018381414

In [7]:
students_passing_math = len(school_data_complete.loc[school_data_complete["math_score"] >= 70]["Student ID"])
percent_passing_math = float(students_passing_math / total_students * 100)
percent_passing_math

74.9808526933878

In [8]:
students_passing_reading = len(school_data_complete.loc[school_data_complete["reading_score"] >= 70]["Student ID"])
percent_passing_reading = float(students_passing_reading / total_students * 100)
percent_passing_reading

85.80546336482001

In [9]:
overall_passing = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)]["Student ID"].count()
percent_overall = float(overall_passing / total_students * 100)
percent_overall

65.17232575950983

In [10]:
#pulling it all into a new dataframe
district_summary_df = pd.DataFrame({"Total Schools":total_schools, 
                                    "Total Students":total_students,
                                    "Total Budget":total_budget,
                                    "Average Math Score":avg_math_score,
                                    "Average Reading Score":avg_reading_score, 
                                    "% Passing Math":percent_passing_math,
                                    "% Passing Reading":percent_passing_reading, 
                                    "% Overall Passing": percent_overall}, index=[0])

#formatting
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("$ {:,.2f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$ 24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [11]:
#group by school name
grouped_by = school_data_complete.set_index('school_name').groupby(['school_name'])

#school type
school_type = school_data.set_index('school_name')['type']

#total students
student_count_per_school = grouped_by['Student ID'].count()

# total school budget
school_budget = school_data.set_index('school_name')['budget']

#per student budget
per_student_budget = school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size']

#avg scores
avg_math = grouped_by['math_score'].mean()
avg_read = grouped_by['reading_score'].mean()

#% passing scores
pass_math = (school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/student_count_per_school)*100
pass_read = (school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/student_count_per_school)*100
overall = (school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/student_count_per_school)*100

#new dataframe
overview_table = pd.DataFrame({
    "School Type": school_type,
    "Total Students": student_count_per_school,
    "Total School Budget": school_budget,
    "Per Student Budget": per_student_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
})

#formatting
overview_table.style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:,.0f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928",$628,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949","$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739","$1,763,916",$644,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468","$917,500",$625,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635","$3,022,020",$652,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087",$581,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917","$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761","$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [12]:
top_5 = overview_table.sort_values("Overall Passing Rate", ascending = False)
top_5.head().style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:,.0f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130",$638,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500",$625,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574",$578,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [13]:
bottom_5 = overview_table.sort_values("Overall Passing Rate", ascending = True)
bottom_5.head().style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:,.0f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363",$637,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020",$652,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [14]:
#by grade
ninth = school_data_complete.loc[school_data_complete["grade"] == "9th"]
tenth = school_data_complete.loc[school_data_complete["grade"] == "10th"]
eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th"]
twelfth = school_data_complete.loc[school_data_complete["grade"] == "12th"]

In [15]:
#avg scores
math_ninth = ninth.groupby("school_name")["math_score"].mean()
math_tenth = tenth.groupby("school_name")["math_score"].mean()
math_eleventh = eleventh.groupby("school_name")["math_score"].mean()
math_twelfth = twelfth.groupby("school_name")["math_score"].mean()

#new dataframe
math_grades = pd.DataFrame({"9th":math_ninth, "10th":math_tenth, 
                             "11th":math_eleventh, "12th":math_twelfth})
math_grades

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [16]:
#avg scores
reading_ninth = ninth.groupby("school_name")["reading_score"].mean()
reading_tenth = tenth.groupby("school_name")["reading_score"].mean()
reading_eleventh = eleventh.groupby("school_name")["reading_score"].mean()
reading_twelfth = twelfth.groupby("school_name")["reading_score"].mean()

#new dataframe
reading_grades = pd.DataFrame({"9th":reading_ninth, "10th":reading_tenth, 
                             "11th":reading_eleventh, "12th":reading_twelfth})
reading_grades

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
#create bins and name for bins
bins = [0, 584, 629, 644, 999]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

In [18]:
school_data_complete["Spending Ranges (Per Student)"] = pd.cut(school_data_complete["budget"]/school_data_complete["size"], bins, labels = group_names)

In [19]:
#creating a group based off of bins
by_spend = school_data_complete.groupby("Spending Ranges (Per Student)")

In [20]:
#avg scores
avg_math_by_spend = by_spend['math_score'].mean()
avg_read_by_spend = by_spend['reading_score'].mean()

#% passing scores
pass_math_by_spend = (school_data_complete[school_data_complete['math_score'] >= 70].groupby('Spending Ranges (Per Student)')['Student ID'].count()/by_spend['Student ID'].count())*100
pass_read_by_spend = (school_data_complete[school_data_complete['reading_score'] >= 70].groupby('Spending Ranges (Per Student)')['Student ID'].count()/by_spend['Student ID'].count())*100
overall_by_spend = (school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('Spending Ranges (Per Student)')['Student ID'].count()/by_spend['Student ID'].count())*100

In [21]:
#new dataframe
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math_by_spend,
    "Average Reading Score": avg_read_by_spend,
    '% Passing Math': pass_math_by_spend,
    '% Passing Reading': pass_read_by_spend,
    "Overall Passing Rate": overall_by_spend
            
})
scores_by_spend

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$584,83.363065,83.964039,93.702889,96.686558,90.640704
$585-629,79.982873,82.312643,79.109851,88.513145,70.939239
$630-644,77.821056,81.301007,70.623565,82.600247,58.841194
$645-675,77.049297,81.005604,66.230813,81.109397,53.528791


## Scores by School Size

* Perform the same operations as above, based on school size.

In [22]:
#create bins and name of bins
bins = [0, 999, 1999, 9999]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]

In [23]:
school_data_complete["School Size"] = pd.cut(school_data_complete["size"], bins, labels = group_name)

In [24]:
#creating a group based off of bins
by_size = school_data_complete.groupby("School Size")

In [25]:
#avg scores
avg_math_by_size = by_size['math_score'].mean()
avg_read_by_size = by_size['reading_score'].mean()

#% passing scores
pass_math_by_size = (school_data_complete[school_data_complete['math_score'] >= 70].groupby('School Size')['Student ID'].count()/by_size['Student ID'].count())*100
pass_read_by_size = (school_data_complete[school_data_complete['reading_score'] >= 70].groupby('School Size')['Student ID'].count()/by_size['Student ID'].count())*100
overall_by_size = (school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('School Size')['Student ID'].count()/by_size['Student ID'].count())*100

In [26]:
#new dataframe
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math_by_size,
    "Average Reading Score": avg_read_by_size,
    '% Passing Math': pass_math_by_size,
    '% Passing Reading': pass_read_by_size,
    "Overall Passing Rate": overall_by_size
            
})
scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (>2000),77.477597,81.198674,68.652380,82.125158,56.574046


## Scores by School Type

* Perform the same operations as above, based on school type

In [27]:
#creating a group based off of bins
by_type = school_data_complete.groupby("type")

In [28]:
#avg scores
avg_math_by_type = by_type['math_score'].mean()
avg_read_by_type = by_type['reading_score'].mean()

#% passing scores
pass_math_by_type = (school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count())*100
pass_read_by_type = (school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count())*100
overall_by_type = (school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count())*100

In [29]:
#new dataframe
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math_by_type,
    "Average Reading Score": avg_read_by_type,
    '% Passing Math': pass_math_by_type,
    '% Passing Reading': pass_read_by_type,
    "Overall Passing Rate": overall_by_type
            
})
scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878
